# Lecture 17 - Random Numbers in Python & Monte Carlo - HW 8

## 1) Random Numbers and Radio Activity

The isotope $^{213}$Bi decays to stable $^{209}$Bi via one of two different routes, with probabilities and
half-lives thus

<img src="Decay9.jpg" alt="Decay process for Bi213 to Bi209" title="Bismuth Decay" />

(Technically, $^{209}$Bi isnt really stable, but it has a half-life of more than $10^{19}$ years, a billion
times the age of the universe, so it might as well be.)

Starting with a sample consisting of 10,000 atoms of $^{213}$Bi, simulate the decay of the atoms
by dividing time into slices of length $\delta t = 1$s each and on each step doing
the following:

1. For each atom of $^{209}$Pb in turn, decide at random, with the appropriate probability, whether it decays or not. (The probability can be calculated from $p(t) = 1 − 2t/\tau $, where $\tau$ is the half life.) Count the total number that decay, subtract it from the number of $^{209}$Pb atoms, and add it to the number of $^{209}$Bi atoms.

2. Now do the same for $^{209}$Tl, except that decaying atoms are subtracted from the total for $^{209}$Tl and added to the total for $^{209}$Pb.

3. For $^{213}$Bi the situation is more complicated: when a $^{213}$Bi atom decays you have to decide at random with the appropriate probability the route by which it decays. Count the numbers that decay by each route and add and subtract accordingly.

Note that you have to work up the chain from the bottom like this, not down from the top,
to avoid inadvertently making the same atom decay twice on a single step.

Keep track of the number of atoms of each of the four isotopes at all times for 20,000 seconds
and make a single graph showing the four numbers as a function of time on the same axes.



In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

# Constants
INITIAL_ATOMS = 10000
TOTAL_TIME = 20000  # seconds
TIME_STEP = 1  # second

# Decay probabilities and half-lives (assuming Bi213 is stable)
decay_probabilities = {
    'Bi212': {'Bi208': 1},
    'Bi208': {'Bi207': 0.5, 'Bi209': 0.5},
}

half_lives = {
    'Bi212': 60.55,  # seconds
    'Bi208': 3.67,   # minutes * 60 seconds
    'Bi207': 31.54,  # days * 24 hours * 3600 seconds
    'Bi209': float('inf'),  # assumed stable
}

def simulate_decay():
    atoms = {isotope: INITIAL_ATOMS for isotope in ['Bi212', 'Bi208', 'Bi207', 'Bi209']}
    times = []
    bi_212_count = atoms['Bi212']
    bi_208_count = atoms['Bi208']
    bi_207_count = atoms['Bi207']
    bi_209_count = atoms['Bi209']

    for t in range(TOTAL_TIME):
        times.append(t)

        # Decay Bi212 to Bi208
        decayed_bi212 = int(bi_212_count * (1 - np.exp(-TIME_STEP / half_lives['Bi212'])))
        bi_212_count -= decayed_bi212
        bi_208_count += decayed_bi212

        # Decay Bi208 to Bi207 or Bi209
        decayed_bi208 = int(bi_208_count * (1 - np.exp(-TIME_STEP / half_lives['Bi208'])))
        bi_208_count -= decayed_bi208
        bi_207_count += decayed_bi208 // 2  # Assuming equal probability for both routes
        bi_209_count += decayed_bi208 // 2

        atoms['Bi212'] = bi_212_count
        atoms['Bi208'] = bi_208_count
        atoms['Bi207'] = bi_207_count
        atoms['Bi209'] = bi_209_count

    return times, atoms

# Run the simulation
times, atoms = simulate_decay()

# Plot the results
plt.figure(figsize=(10, 6))
for isotope in ['Bi212', 'Bi208', 'Bi207', 'Bi209']:
    plt.plot(times, atoms[isotope], label=f'{isotope} count')

plt.xlabel('Time (seconds)')
plt.ylabel('Number of Atoms')
plt.title('Decay Chain Simulation of Bi213')
plt.legend()
plt.grid(True)
plt.show()


## 2) Lets Make a Deal
Monte Carlo methods are often useful to ensure that our thinking is reasonable. A good
example of this kind of use is to investigate a simple problem that generated much attention
several years ago and for which many mathematicians obtained an incorrect solution.

The problem was the analysis of the optimal strategy in a television game show popular at
the time. The show was Lets Make a Deal with host Monty Hall. At some point in the show,
a contestant was given a choice of selecting one of three possible items, each concealed behind
one of three closed doors. The items varied considerably in value.

After the contestant made a choice but before the chosen door was opened, the
host, who knew where the most valuable item was, would open one of the doors
not selected and reveal a worthless item.

**The host would then offer to let the contestant select a different door from what was originally
selected. The question, of course, is should the contestant switch?**

<img src="Monty_Hall_Problem.jpg" alt="Decay process for Bi213 to Bi209" title="Bismuth Decay" />



Much interest in this problem was generated when it was analyzed by a popular magazine
writer, Marilyn vos Savant, who concluded that the optimal strategy is to switch. This
strategy is counterintuitive to many mathematicians, who would say that there is nothing to
be gained by switching; that is, that the probability of improving the selection is 0.5. Study
this problem by Monte Carlo methods. Be careful to understand all of the assumptions

**Write a code that implement this test for 1000 “games”, 500 where the contestant choose to KEEP their choice of door, and 500 where contestant chooses to CHANGE their choice of door:**


## Determine if there is probability of improving the selection by switching, and if so by how much?



In [3]:
import random

def monty_hall_simulation(num_games):
    # Initialize counters
    keep_wins = 0
    switch_wins = 0
    
    for _ in range(num_games // 2):
        # Randomly place the car behind one of three doors
        car_door = random.randint(1, 3)
        
        # Contestant's initial choice
        choice = random.randint(1, 3)
        
        # Host opens a door with a goat
        open_doors = set(range(1, 4)) - {car_door} - {choice}
        host_opens = random.choice(list(open_doors))
        
        # Check if contestant wins by keeping the choice
        if choice == car_door:
            keep_wins += 1
        
        # Check if contestant wins by switching
        switch_choice = min(set(range(1, 4)) - {choice} - {host_opens})
        if switch_choice == car_door:
            switch_wins += 1
    
    return keep_wins / num_games, switch_wins / num_games

# Run the simulation
num_games = 1000
keep_probability, switch_probability = monty_hall_simulation(num_games)

print(f"Probability of winning by keeping the choice: {keep_probability:.4f}")
print(f"Probability of winning by switching: {switch_probability:.4f}")

# Calculate improvement in switching strategy
improvement = switch_probability - keep_probability
print(f"Improvement in probability by switching: {improvement:.4f}")

Probability of winning by keeping the choice: 0.1690
Probability of winning by switching: 0.3310
Improvement in probability by switching: 0.1620
